Libraries

In [4]:
#General
import os
import time
import csv
import re
import random

#Database Connection
import mysql.connector

#Data Analysis and scraping
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

Prepare Database

In [2]:
#Connect to the database
db = mysql.connector.connect(
    host="localhost",
    user="user",
    password="password",
    port="3306",
    database="realestatepredictor"
)

#Check if the table homegate exists. Set up the table if it does not exist
mycursor = db.cursor()
mycursor.execute("SHOW TABLES")
tables = mycursor.fetchall()
if ('homegate',) in tables:
    print("Table homegate exists")
else:
    print("Table homegate does not exist. Setting up table.")
    mycursor.execute("CREATE TABLE homegate (id INT AUTO_INCREMENT PRIMARY KEY, title VARCHAR(255), price FLOAT, rooms FLOAT, floor INT, address VARCHAR(255), url VARCHAR(255))")

InterfaceError: 2003: Can't connect to MySQL server on 'localhost:3306' (10061 No connection could be made because the target machine actively refused it)

Homegate

In [5]:
# Liste mit User-Agents für Rotation (if needed)
ua_path = "user_agents.txt"
ua_list = [line.rstrip('\n') for line in open(ua_path)]
ua_list[:5] # Show first 5 user agents

# Chrome-Driver
opts = webdriver.ChromeOptions()
opts.add_argument("user-agent=" + random.choice(ua_list)) # Rotate user agent
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=opts) # Start Chrome-Driver

# Website to scrape
url = 'https://www.homegate.ch/mieten/immobilien/kanton-zuerich/trefferliste' # URL for homegate.ch
driver.get(url) # Open website

# Get the full content of the website
source = driver.page_source # Get page source

# Close driver
driver.close() # Close driver

# Parse HTML content with BeautifulSoup
soup = BeautifulSoup(source, 'html.parser') # Parse HTML content

# Initialize lists for data
prices = []
rooms = []
floor_areas = []
addresses = []
coordinates = []

# Assuming 'listing-item' is the class for property listings
for listing in soup.find_all(class_='listing-item'):
    # Extract price
    price = listing.find(class_='price').text.strip()
    prices.append(price)
    
    # Extract number of rooms
    room = listing.find(class_='room').text.strip()
    rooms.append(room)
    
    # Extract floor area
    floor_area = listing.find(class_='area').text.strip()
    floor_areas.append(floor_area)
    
    # Extract address
    address = listing.find(class_='address').text.strip()
    addresses.append(address)
    
    # Extract coordinates if available
    # You'll need to find the correct class or script that contains the coordinates
    # This is just a placeholder example
    coord_script = listing.find('script', type='application/ld+json')
    if coord_script:
        coord_data = json.loads(coord_script.text)
        coordinates.append((coord_data['latitude'], coord_data['longitude']))
    else:
        coordinates.append((None, None))

# Create a DataFrame
df = pd.DataFrame({
    'Price': prices,
    'Rooms': rooms,
    'Floor Area': floor_areas,
    'Address': addresses,
    'Coordinates': coordinates
})

# Save to file
# df.to_csv('homegate_data.csv', sep=";", index=False) # Save to file

print(df.head()) # Print first few rows of the DataFrame

Empty DataFrame
Columns: [Price, Rooms, Floor Area, Address, Coordinates]
Index: []
